## Exercício: Modelo de Linguagem com auto-atenção e máscaras causais

Seguimos na mesma linha de treinar um modelo de linguagem a partir dos textos do livro "O Guarani", de José de Alencar.

Neste exercício, vamos treinar um modelo de linguagem com auto-atenção e com máscara causal. A máscara causal é necessária para que o modelo não tenha acesso a palavras futuras, que é a abordagem usada por grandes modelos de linguagem, como o GPT.

Use a implementação matricial de auto-atenção da aula passada.

### Modificações necessárias

* Adicione a máscara causal na função `forward` da cabeça de auto-atenção.
* Modifique o nosso dataloader para retornar inputs (uma lista de tokens de tamanho $n$), targets (uma lista de tokens de tamanho $n$ deslocada para a esquerda em 1 token). Exemplo `input = [1, 2, 3, 4]`, `target = [2, 3, 4, 5]` para a sequência `[1, 2, 3, 4, 5]` com `seq_len=4`, por exemplo (Ver slide 50).

### Extra
* Multiembedding_dimAttention: modifique a cabeça de auto-atenção para ter múltiplas cabeças. Isso não é obrigatório, mas pode ser interessante para ver como o modelo se comporta.
* Diagrama da geração: fazer diagrama que mostre os passos da geração de tokens (conforme slide 47).

### Dicas

* Use como base o vídeo do Karpathy: https://www.youtube.com/watch?v=kCc8FmEb1nY. Observe que, no vídeo, ele primeiro implementa um modelo bi-grama, depois um modelo de linguagem com auto-atenção. O modelo de auto-atenção é implementado por volta do minuto 40, mas vale a pena assistir o vídeo todo.
* Use esta implementação como base: https://colab.research.google.com/drive/1vFTg4MSXVJwNSzPjaCcvmqhxTP7gK7HA?usp=sharing. Observe como o modelo é organizado e como a máscara é implementada na classe MultiHeadAttention.
* Use `context_size=9`

## Imports

In [1]:
import os
import sys
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import math
import re
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import Dataset, DataLoader
from collections import Counter

## Variáveis Globais e Inicialização

In [2]:
# Global variables

# Vocabulary
vocab_size = 10000
seq_len = 9
pattern = r'\w+|[,;.:!?\']'

# Training
batch_size = 128
epochs = 10
lr = 0.1

# Model
embedding_dim = 256
hidden_dim = 128
dropout_rate = 0.2

# Colab environment
IN_COLAB = 'google.colab' in sys.modules

if (IN_COLAB):
    %pip install colorama

    # Google Drive
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    project_folder="/content/drive/MyDrive/Classes/IA024/Aula_2_3"
    os.chdir(project_folder)
    !ls -la

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Faz download e carrega o dataset

In [3]:
# Check if download is necessary
if not os.path.exists("67724.txt.utf-8"):
    print("Downloading Gutenberg texts")

    !wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
    !wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

In [4]:
text = open("67724.txt.utf-8","r").read()
text += open("67725.txt.utf-8","r").read()

paragraphs = text.split("\n\n")

len(paragraphs)

4969

In [5]:
# Checking the text
print(paragraphs[0])

The Project Gutenberg eBook of O Guarany: romance brazileiro, Vol. 1 (of 2)
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.


In [6]:
cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]

# Print 5 random paragraphs
num_paragraphs = len(cleaned_paragraphs)
for i in range(0,5):
    idx = random.randrange(num_paragraphs)
    print(f"{cleaned_paragraphs[idx]}\n")

print("Number of paragraphs: " + str(num_paragraphs))

len(cleaned_paragraphs)

LEALDADE

 FIM DAS NOTAS DO TOMO PRIMEIRO.

E tomando tres flechas, esticou a corda do arco; mataria os aventureiros sem que um percebesse a morte dos outros.

--Pery quer salvar a senhora.

O que se passava era para elle uma cousa incomprehensivel, um enigma indecifravel para quem ignorava o que succedêra anteriormente.

Number of paragraphs: 4892


4892

## Análise do dataset

In [7]:
# Conta as palavras no dataset
def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(r'\w+', text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

12603

## Criando um vocabulário

In [8]:
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}

In [9]:
def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(pattern, sentence.lower())]

print(cleaned_paragraphs[20])
print(encode_sentence(cleaned_paragraphs[20], vocab))

 Publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
[6594, 139, 4376, 19, 6595, 0, 6, 44, 110, 269, 259, 2662, 10, 1064, 6596, 0, 2, 186, 130, 280, 3, 2257, 6, 6597, 1, 2665, 0]


## Classe do dataset

### Dataset Modificado para Máscara Causal

Modifique o nosso dataloader para retornar inputs (uma lista de tokens de tamanho $n$), targets (uma lista de tokens de tamanho $n$ deslocada para a esquerda em 1 token). Exemplo `input = [1, 2, 3, 4]`, `target = [2, 3, 4, 5]` para a sequência `[1, 2, 3, 4, 5]` com `seq_len=4`, por exemplo (Ver slide 50).

In [10]:
# Dataset class
import torch
from torch.utils.data import Dataset, DataLoader

class CausalMaskDataset(Dataset):
  def __init__(self, paragraphs, vocab, seq_len):
    self.paragraphs = paragraphs
    self.vocab = vocab
    self.seq_len = seq_len
    self.tokens, self.targets = self.setup()

  def __len__(self):
    return len(self.tokens)

  def __getitem__(self, idx):
    tokens = torch.tensor(self.tokens[idx])
    targets = torch.tensor(self.targets[idx])
    return tokens, targets
  
  def setup(self):
    tokens = []
    targets = []
    for paragraph in self.paragraphs:
      encoded = encode_sentence(paragraph, self.vocab)
      
      # If paragraph is smaller than the sequence length, skip it.
      if len(encoded) < self.seq_len + 1:
          continue

      for i in range(len(encoded) - self.seq_len):
        tks = encoded[i:i+self.seq_len]
        # Return targets with seq_len instead of a single one.
        tgt = encoded[i+1:i+1+self.seq_len]
        # Only add if there are no unknown tokens in both context and target.
        bad_token = 0
        if not (bad_token in tks or tgt == bad_token):
          tokens.append(tks)
          targets.append(tgt)
    return tokens, targets

#### Verificação básica do Dataset modificado

In [11]:
tst_paragraphs = cleaned_paragraphs[:50]
tst_seq_len = 9
tst_dataset = CausalMaskDataset(tst_paragraphs, vocab, tst_seq_len)

print("Input and Target Tensors:")
tst_dataset[0]


Input and Target Tensors:


(tensor([133,  47, 712, 144, 537, 324, 275,  35, 499]),
 tensor([ 47, 712, 144, 537, 324, 275,  35, 499,  47]))

### Verificação do Dataset

In [12]:
# Train/Validation split
train_data, val_data = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)

train_dataset = CausalMaskDataset(train_data, vocab, seq_len)
val_dataset = CausalMaskDataset(val_data, vocab, seq_len)

# Counting all Samples
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print()
print(f"Training dataset samples: {len(train_dataset)}")
print(f"Validation dataset samples: {len(val_dataset)}")

Training samples: 3913
Validation samples: 979

Training dataset samples: 15993
Validation dataset samples: 4197


In [13]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

sample = next(iter(train_loader))
print(sample)

[tensor([[  13,    2,    6,  ..., 1866,   11,  150],
        [   4, 2248,    5,  ..., 6513,  298,  117],
        [1996,   22,  155,  ...,  591, 1785,   11],
        ...,
        [ 459,   55,   32,  ...,  961,    7, 2140],
        [   5, 9934,    2,  ...,   72,   15,  300],
        [  79,   35,   67,  ...,  304,  325, 1158]]), tensor([[   2,    6, 8530,  ...,   11,  150,    0],
        [2248,    5,    4,  ...,  298,  117,   40],
        [  22,  155,  104,  ..., 1785,   11,  637],
        ...,
        [  55,   32,    1,  ...,    7, 2140,    2],
        [9934,    2, 1207,  ...,   15,  300,    0],
        [  35,   67,   59,  ...,  325, 1158, 5986]])]


## Modelo Matricial

### Embedding Posicional

In [14]:
# Positional Embedding - as described in "Attention is All You Need"
class PositionalEncoding(nn.Module):
    def __init__(self, max_sequence, embedding_dim):
        super().__init__()
        self.positional_encoding = torch.zeros(max_sequence, embedding_dim, device=device)
        position = torch.arange(0, max_sequence, device=device).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2, device=device) * (-math.log(10000.0) / embedding_dim))
        self.positional_encoding[:, 0::2] = torch.sin(position * div_term)
        self.positional_encoding[:, 1::2] = torch.cos(position * div_term)
        self.positional_encoding = self.positional_encoding.unsqueeze(0)

    def forward(self, x):
        _, seq_length, _ = x.size()
        positional_encoding = self.positional_encoding[:, :seq_length, :]
        positional_encoding = positional_encoding.to(x.device)
        # Position encoding is added to the input embeddings.
        return x + positional_encoding   

### Atenção Matricial com Máscara Causal

In [15]:
# Matrix Implementation (Single Head)
class SelfAttention_Matrix(nn.Module):
  def __init__(self, embedding_dim, seq_len):
    super().__init__()

    self.WQ = nn.Linear(embedding_dim, embedding_dim, bias=False)
    self.WK = nn.Linear(embedding_dim, embedding_dim, bias=False)
    self.WV = nn.Linear(embedding_dim, embedding_dim, bias=False)
    self.WO = nn.Linear(embedding_dim, embedding_dim, bias=False)
    self.seq_len = seq_len
    self.register_buffer('tril', torch.tril(torch.ones(self.seq_len, self.seq_len)))

  def forward(self, inputs):
    # Linear projections
    Q = self.WQ(inputs)
    K = self.WK(inputs)
    V = self.WV(inputs)

    scores = torch.matmul(Q, K.transpose(-2, -1)) # B,Seq,Seq
    scores = scores.masked_fill(self.tril[:self.seq_len, :self.seq_len] == 0, float('-inf'))

    probs = F.softmax(scores, dim=-1)
    new_embedding = torch.matmul(probs, V)
    # Projection in WO
    new_embedding = self.WO(new_embedding)
    return new_embedding

#### Classe do Modelo Matricial

In [16]:
class LanguageModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, seq_len, h):
        super(LanguageModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size+1, embedding_dim)
        self.posencoding = PositionalEncoding(seq_len, embedding_dim)
        self.attention = SelfAttention_Matrix(embedding_dim, seq_len)        
        self.linear1 = nn.Linear(seq_len * embedding_dim, h)
        self.dropout_rate1 = nn.Dropout(dropout_rate)
        self.relu = torch.nn.ReLU()
        self.linear2 = nn.Linear(h, vocab_size+1)
        self.dropout_rate2 = nn.Dropout(dropout_rate)
        # Softmax to scale outputs
        self.logSoftMax = torch.nn.LogSoftmax(dim=1)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        embeds_pos = self.posencoding(embeds)
        x = torch.stack(torch.unbind(embeds_pos, dim=1), dim=1)
        # Camada de autoatenção
        attention  = self.attention(x)
        # Flatten embeddings
        embeds = embeds.view(attention.size(0), -1)
        # Linear layer
        out = self.linear1(embeds)
        out = self.dropout_rate1(out)
        out = self.relu(out)
        # Second layer
        out = self.linear2(out)
        out = self.dropout_rate2(out)
        # Softmax output
        out = self.logSoftMax(out)
        return out

## Treinamento e Avaliação

### Funções de Treinamento e Avaliação

#### Contagem de Parâmetros do Modelo

In [17]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'The model has a total of {total_params:,} parameters.')

#### Avaliação Inicial Pré-Treinamento

In [18]:
def initial_eval(model, criterion, device):
    # Initial Perplexity and Loss
    # Before training
    model.eval()

    loss = 0
    perp = 0

    with torch.no_grad():
        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
    loss /= len(train_loader)
    perp = torch.exp(torch.tensor(loss))

    print(f'Initial Loss: {loss:.4f}')
    print(f'Initial Perplexity: {perp:.4f}')

#### Função de Treinamento

In [19]:
def train(model, criterion, optimizer, device):
      # Training Loop
      model.train()
      for epoch in range(epochs):

            epoch_start = time.time()
            # Metrics
            epoch_loss = 0
            epoch_correct = 0
            epoch_samples = 0

            for inputs, targets in train_loader:
                  inputs = inputs.to(device)  # Move input data to the device
                  targets = targets.to(device)

                  # Forward pass
                  outputs = model(inputs)
                  loss = criterion(outputs, targets)

                  # Backward pass and optimization
                  optimizer.zero_grad()
                  loss.backward()
                  optimizer.step()

                  # Loss
                  epoch_loss += loss.item()

                  # Predicted
                  _, predicted = torch.max(outputs, 1)
                  epoch_correct += (predicted == targets).sum().item()
                  epoch_samples += targets.size(0)

            # Calculate average loss and accuracy for epoch
            avg_loss = epoch_loss / len(train_loader)
            acc = epoch_correct / epoch_samples

            # Perplexity
            perp = torch.exp(torch.tensor(avg_loss))

            epoch_end = time.time()
            epoch_time = epoch_end - epoch_start
            # Print epoch statistics
            print(f'Epoch [{epoch+1}/{epochs}], Time:{epoch_time:.2f}, Loss: {avg_loss:.4f}, Accuracy: {acc:.2f}%, Perplexity: {perp:.4f}')


#### Função de Avaliação do Modelo Treinado

In [20]:
def eval(model, criterion, device):
    model.eval()

    loss_sum = 0
    total_sum = 0
    correct_sum = 0
    eval_round = 0

    loss = 0
    perp = 0

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)      
            loss_sum += loss

            # Get the predicted labels
            _, predicted = torch.max(outputs, 1)

            total_sum += targets.size(0)
            correct_sum += (predicted == targets).sum().item()
            eval_round += 1

    # Calculate accuracy
    acc = 100 * correct_sum / total_sum

    # Calculate average perplexity
    average_loss = loss_sum / len(val_loader)
    average_perplexity = torch.exp(average_loss)

    print(f'Test Accuracy: {acc:.2f}%')
    print(f'Average Loss: {average_loss:.2f}')
    print(f'Average Perplexity: {average_perplexity:.2f}')

### Avaliação Inicial

In [21]:
model_attn = LanguageModel(vocab_size, embedding_dim, seq_len, hidden_dim)
print("Model with Self Attention:")
print()
count_parameters(model_attn)

# Cross Entropy
criterion = nn.CrossEntropyLoss()

model_attn.to(device)

print()
print("Initial Evaluation")
print()
initial_eval(model_attn, criterion, device)

Model with Self Attention:

The model has a total of 4,407,569 parameters.

Initial Evaluation



RuntimeError: 0D or 1D target tensor expected, multi-target not supported

### Treinamento do Modelo

In [ ]:
model_attn = LanguageModel(vocab_size, embedding_dim, seq_len, hidden_dim)
print("Model with Self Attention:")
print()
count_parameters(model_attn)

# Cross Entropy
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model_attn.parameters(), lr)

model_attn.to(device)

print()
print("Training Start")
print()
train(model_attn, criterion, optimizer, device)

### Avaliação do Modelo

In [ ]:
print()
print("Evaluation Start")
print()
eval(model_attn, criterion, device)

## Exemplo de uso

In [ ]:
# Código adaptado da implementação do Cesar Bastos
from colorama import Fore, Style

text = cleaned_paragraphs
model_attn.to(device)
def generate_text(model, vocab, text, max_length, context_size):
    words = []
    # Ensure there are enough words for at least one sequence
    while len(words) < context_size:
        random_number = random.randint(1, 4891)
        words = encode_sentence(text[random_number], vocab)
        if not words:
            words = []
            continue  # Skip if the sentence cannot be encoded
        words = words[:context_size]
        #print(words)
        if any(token == 0 for token in words):
            words = []
            continue  # Skip if any token is zero (assuming 0 is a special token)
        context = words

    print(f"Frase: {cleaned_paragraphs[random_number]}")
    print(words)

    for _ in range(max_length):
        words_tensor = torch.tensor(context[-context_size:], dtype=torch.long).unsqueeze(0).to(device)
        logits = model(words_tensor)
        probs = F.softmax(logits, dim=1)
        next_token = torch.multinomial(probs, num_samples=1)
        context.append(next_token.item())
        print(context)
    frase = []
    for i in context: ##Agradecimentos a Ramon Abilio
        word = next((word for word, code in vocab.items() if code == i), "<UNKNOWN>")
        frase.append(word)

    print(f"{Fore.BLUE}{frase[:context_size]}{Style.RESET_ALL} {Fore.RED}{frase[-max_length:]}{Style.RESET_ALL} ")


max_length= 10
generate_text(model_attn, vocab, text, max_length, context_size)